The Models have been trained using our own GPU; now we will calculate the mAP and show top 10 images with some more metrics.                                Models are stored in google drive due to size.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
import os
from sklearn.model_selection import train_test_split

folder_path = "/content/VOCdevkit/VOC2008/JPEGImages"
def extract_positive_images(directory, file_name):
    positive_images = []
    variations = [
        file_name,
        file_name.replace("_train", "_trainval"),
        file_name.replace("_train", "_val")
    ]

    for variation in variations:
        file_path = os.path.join(directory, variation)
        with open(file_path, "r") as file:
            lines = file.readlines()

        for line in lines:
            if line.strip().endswith(" 1"):
                image_name = line.split()[0] + ".jpg"
                if image_name not in positive_images:
                    positive_images.append(image_name)

    return positive_images


data_directory = "VOCdevkit/VOC2008/ImageSets/Main/"
folder_path = "/VOCdevkit/VOC2008/JPEGImages"

aeroplane_images = extract_positive_images(data_directory, "aeroplane_train.txt")
bicycle_images = extract_positive_images(data_directory, "bicycle_train.txt")
bird_images = extract_positive_images(data_directory, "bird_train.txt")
boat_images = extract_positive_images(data_directory, "boat_train.txt")
bottle_images = extract_positive_images(data_directory, "bottle_train.txt")
bus_images = extract_positive_images(data_directory, "bus_train.txt")
car_images = extract_positive_images(data_directory, "car_train.txt")
cat_images = extract_positive_images(data_directory, "cat_train.txt")
chair_images = extract_positive_images(data_directory, "chair_train.txt")
cow_images = extract_positive_images(data_directory, "cow_train.txt")
diningtable_images = extract_positive_images(data_directory, "diningtable_train.txt")
dog_images = extract_positive_images(data_directory, "dog_train.txt")
horse_images = extract_positive_images(data_directory, "horse_train.txt")
motorbike_images = extract_positive_images(data_directory, "motorbike_train.txt")
person_images = extract_positive_images(data_directory, "person_train.txt")
pottedplant_images = extract_positive_images(data_directory, "pottedplant_train.txt")
sheep_images = extract_positive_images(data_directory, "sheep_train.txt")
sofa_images = extract_positive_images(data_directory, "sofa_train.txt")
train_images = extract_positive_images(data_directory, "train_train.txt")
tvmonitor_images = extract_positive_images(data_directory, "tvmonitor_train.txt")

jpg_files = []

for file in os.listdir(folder_path):
    if file.endswith(".jpg"):
        jpg_files.append(file)


class_data_lists = {
    "Aeroplane": aeroplane_images,
    "Bicycle": bicycle_images,
    "Bird": bird_images,
    "Boat": boat_images,
    "Bottle": bottle_images,
    "Bus": bus_images,
    "Car": car_images,
    "Cat": cat_images,
    "Chair": chair_images,
    "Cow": cow_images,
    "Diningtable": diningtable_images,
    "Dog": dog_images,
    "Horse": horse_images,
    "Motorbike": motorbike_images,
    "Person": person_images,
    "Pottedplant": pottedplant_images,
    "Sheep": sheep_images,
    "Sofa": sofa_images,
    "Train": train_images,
    "Tvmonitor": tvmonitor_images
}


train_data_lists = {class_name: [] for class_name in class_data_lists}
test_data_lists = {class_name: [] for class_name in class_data_lists}


for class_name, data_list in class_data_lists.items():
    train_images, test_images = train_test_split(data_list, test_size=0.2, random_state=42)
    train_data_lists[class_name] = train_images
    test_data_lists[class_name] = test_images


for class_name, train_images in train_data_lists.items():
    for file in train_images:
        try:
            src_path = os.path.join(folder_path, file)
            dest_dir = os.path.join("train", class_name)
            dest_path = os.path.join(dest_dir, file)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copyfile(src_path, dest_path)
        except Exception as e:
            print(f"Error copying {file} to {dest_dir}: {e}")


for class_name, test_images in test_data_lists.items():
    for file in test_images:
        try:
            src_path = os.path.join(folder_path, file)
            dest_dir = os.path.join("test", class_name)
            dest_path = os.path.join(dest_dir, file)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copyfile(src_path, dest_path)
        except Exception as e:
            print(f"Error copying {file} to {dest_dir}: {e}")



In [ ]:
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set the paths to the train and test directories
train_folder_path = "train"

# Each class has its own directory within the train directory
class_names = [d for d in os.listdir(train_folder_path) if os.path.isdir(os.path.join(train_folder_path, d))]

# Create a dictionary to hold our training image paths
train_data_lists = {class_name: [os.path.join(train_folder_path, class_name, img)
                                 for img in os.listdir(os.path.join(train_folder_path, class_name))]
                    for class_name in class_names}

def build_binary_classification_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Configure the ImageDataGenerator for training data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Loop through each class and train a binary classification model
for class_name in class_names:
    print(f"Training for class: {class_name}")

    # Retrieve the list of image paths for the positive class
    positive_images = train_data_lists[class_name]
    if not positive_images:
        print(f"No images found for class {class_name}. Skipping this class.")
        continue
    positive_labels = [1] * len(positive_images)

    # Build a list of image paths for the negative class (all other classes)
    negative_images = []
    for other_class_name, image_paths in train_data_lists.items():
        if other_class_name != class_name:
            negative_images.extend(image_paths)

    print(f"Number of positive samples for {class_name}: {len(positive_images)}")
    print(f"Number of negative samples before balancing: {len(negative_images)}")


    random.shuffle(negative_images)  # Shuffle the negative images
    negative_images = negative_images[:len(positive_images)]  # Balance the positive and negative datasets
    negative_labels = [0] * len(negative_images)
    print(f"Number of negative samples after balancing: {len(negative_images)}")

    # Combine and shuffle the positive and negative samples
    combined_images = positive_images + negative_images
    combined_labels = positive_labels + negative_labels
    combined_list = list(zip(combined_images, combined_labels))
    random.shuffle(combined_list)
    combined_images, combined_labels = zip(*combined_list)

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(combined_images, combined_labels, test_size=0.2, random_state=42)

    # Create DataFrames for the training and validation sets
    train_df = pd.DataFrame({
    'filename': X_train,
    'label': [str(label) for label in y_train]  # Convert labels to strings
})
    val_df = pd.DataFrame({
    'filename': X_val,
    'label': [str(label) for label in y_val]  # Convert labels to strings
})

    # Create data generators for training and validation
    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        x_col='filename',
        y_col='label',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
    )

    val_datagen = ImageDataGenerator(rescale=1./255)
    val_generator = val_datagen.flow_from_dataframe(
        val_df,
        x_col='filename',
        y_col='label',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
    )

    # Build the binary classification model
    model = build_binary_classification_model()

    # Train the model on the data
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // 32,
        validation_data=val_generator,
        validation_steps=len(X_val) // 32,
        epochs=50
    )

    # Save the trained model
    model_save_path = os.path.join('models', f"{class_name}_binary_model.h5")
    os.makedirs('models', exist_ok=True)
    model.save(model_save_path)

    print(f"Finished training for class: {class_name}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = '/content/test.zip'

# Extract to a folder named 'test_data'
extract_folder = '/content/test_data'

# Create directory if it doesn't exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Extraction Completed")

Extraction Completed


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont


In [ ]:
# Paths to models and test data
model_dir = '/content/drive/My Drive/models'
test_data_dir = '/content/test_data/test'

# Initialize ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)


def display_image_with_prediction(image_path, prediction, confidence):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", size=20)
    except IOError:
        font = ImageFont.load_default()
    text = f"{prediction}, {confidence:.2f}"
    draw.text((10, 10), text, fill="red", font=font)
    display(image)


In [ ]:
model_dir = '/content/drive/My Drive/models'
test_data_dir = '/content/test_data/test'

# Initialize ImageDataGenerator for preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Store average precision scores for mAP calculation
ap_scores = []

# Loop through each model file in the models directory
for model_file in sorted(os.listdir(model_dir)):
    if model_file.endswith(".h5"):
        print(f"\nLoading model {model_file}...")
        model_path = os.path.join(model_dir, model_file)
        model = load_model(model_path)
        class_name = model_file.replace("_binary_model.h5", "")
        print(f"Model for class '{class_name}' loaded successfully.")

        # Create DataFrame for test data with labels as strings
        images = []
        labels = []
        for folder in os.listdir(test_data_dir):
            folder_path = os.path.join(test_data_dir, folder)
            for image_file in os.listdir(folder_path):
                images.append(os.path.join(folder, image_file))
                labels.append(str(int(folder == class_name)))  # Convert boolean to int to string

        test_df = pd.DataFrame({
            'filename': images,
            'label': labels  # Labels are now strings
        })

        # Prepare test generator
        test_generator = test_datagen.flow_from_dataframe(
            dataframe=test_df,
            directory=test_data_dir,
            x_col='filename',
            y_col='label',
            target_size=(224, 224),
            batch_size=32,
            class_mode='binary',
            shuffle=False)

        # Predict and evaluate
        predictions = model.predict(test_generator, steps=np.ceil(len(test_df)/32))
        predicted_labels = (predictions > 0.5).astype(int)
        ap_score = average_precision_score(test_generator.classes, predictions)
        ap_scores.append(ap_score)
        print(f"AP score for class '{class_name}': {ap_score:.3f}")


        # Display top 10 images based on prediction scores
        top_indices = np.argsort(predictions[:, 0])[::-1][:10]  # Indices of top 10 predictions
        for idx in top_indices:
            image_path = test_df.iloc[idx]['filename']
            actual_label = "Positive" if test_df.iloc[idx]['label'] == '1' else "Negative"
            predicted_confidence = predictions[idx, 0]
            display_image_with_prediction(os.path.join(test_data_dir, image_path), actual_label, predicted_confidence)


        # Display metrics and confusion matrix
        print("\nClassification Report:")
        print(classification_report(test_generator.classes, predicted_labels))
        print("Confusion Matrix:")
        print(confusion_matrix(test_generator.classes, predicted_labels))

# Calculate mean Average Precision (mAP)
mAP = np.mean(ap_scores)
print(f"\nMean Average Precision (mAP) across all classes: {mAP:.3f}")